In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('./train.csv', index_col='Id')
X_test_full = pd.read_csv('./test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [3]:
X_train.head()

MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
Id                                                                           
619          20         90.0    11694            9            5       2007   
871          20         60.0     6600            5            5       1962   
93           30         80.0    13360            5            7       1921   
818          20          NaN    13265            8            5       2002   
303          20        118.0    13704            7            5       2001   

     YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  ...  GarageArea  \
Id                                                     ...               
619          2007       452.0          48           0  ...         774   
871          1962         0.0           0           0  ...         308   
93           2006         0.0         713           0  ...         432   
818          2002       148.0        1218           0  ...         857   
303          2002       150.0           0           0  ...         843   

     WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  \
Id                                                                              
619           0          108              0          0          260         0   
871           0            0              0          0            0         0   
93            0            0             44          0            0         0   
818         150           59              0          0            0         0   
303         468           81              0          0            0         0   

     MiscVal  MoSold  YrSold  
Id                            
619        0       7    2007  
871        0       8    2009  
93         0       8    2009  
818        0       7    2008  
303        0       1    2006  

[5 rows x 36 columns]

In [4]:
# Step 1 : Preliminary investigation #

In [13]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [14]:
# Part A#

In [15]:
# Fill in the line below: How many rows are in the training data?
num_rows = 1168

# Fill in the line below: How many columns in the training data
# have missing values?
num_cols_with_missing = 3

# Fill in the line below: How many missing entries are contained in 
# all of the training data?
tot_missing = 212 + 6 + 58

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
#Step 2 : Drop columns with missing data#

In [17]:
# Fill in the line below: get names of columns with missing values
columns_with_missing_values = [col for col in X_train.columns if X_train[col].isnull().any()]

# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(columns_with_missing_values , axis = 1 )
reduced_X_valid = X_valid.drop(columns_with_missing_values , axis = 1 )


In [18]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


In [ ]:
# Step 3 : Imputition #

In [19]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
my_imputition = SimpleImputer() 

imputed_X_train = pd.DataFrame(my_imputition.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputition.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns


In [20]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


In [21]:
# Step 4: Generate test predictions #

In [22]:
# Preprocessed training and validation features
final_X_train = imputed_X_train
final_X_valid = imputed_X_valid

In [23]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
18062.894611872147


In [24]:
# Part B #

In [25]:
# Fill in the line below: preprocess test data
final_X_test = pd.DataFrame(my_imputition.transform(X_test))

# Fill in the line below: get test predictions
preds_test = model.predict(final_X_test)


C:\Users\pc\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [26]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)
